# Building Spam Filter using Naive Bayes

To classify messages as spam or non-spam:
<ul>
<li>Learns how humans classify messages.
<li>Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
<li>Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).
</ul>

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning Repository.

## Exploring dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# dataset
spam_collection = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=['Label', 'SMS'])

spam_collection.shape

(5572, 2)

In [3]:
spam_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam_collection["Label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

There are about 87% ham(non-spam messages) and remaining 13% contains spam messages. 

## Training and Test set

When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:
<ul>
<li>A training set, which we'll use to "train" the computer how to classify messages.
<li>A test set, which we'll use to test how good the spam filter is with classifying new messages.
</ul>

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

<ul>
<li>The training set will have 4,458 messages (about 80% of the dataset).
<li>The test set will have 1,114 messages (about 20% of the dataset).
</ul>

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

In [5]:
# Randomize entire dataset
randomized_data = spam_collection.sample(frac=1, random_state=1)

# Get split index
split_index = round(len(randomized_data) * 0.8)

# Assigning training and test data
training_data = randomized_data[:split_index].reset_index(drop=True)
test_data = randomized_data[split_index:].reset_index(drop=True)

print(training_data.shape)
print(test_data.shape)

(4458, 2)
(1114, 2)


In [6]:
# Get counts of spam and ham from training and test data set
training_data["Label"].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test_data["Label"].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The training set accounts for 80% of the dataset, and the remaining 20% of the data is the test set.

## Letter case and punctuation

We'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

In [8]:
# Before cleaning
training_data.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
# After cleaning
training_data["SMS"] = training_data["SMS"].str.replace(r'\W+', ' ').str.lower()
training_data.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on da...


## Creating the Vocabulary

With the exception of the "Label" column, every other column in the transformed table above represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message). 
We'll eventually bring the training set to that format ourselves, but first, let's create a list with all of the unique words that occur in the messages of our training set.

In [10]:
training_data["SMS"] = training_data["SMS"].str.split()
vocabulary = []

for message in training_data["SMS"]:
    vocabulary.append(message)

vocabulary = list(set(sum(vocabulary, [])))
len(vocabulary)


7783

There are 7783 words in the SMS messages

## Final Training Set

Eventually, we're going to create a new DataFrame. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

In [11]:
# Dataframe for training set vocabulary lists

word_counts_per_sms = {unique_word: [0] * len(training_data['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [12]:
training_data = pd.concat([training_data, word_counts], axis=1)
training_data.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:
<ul>
<li>P(Spam) and P(Ham)
<li>NSpam, NHam, NVocabulary
</ul>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [13]:
# Calulate alpha
alpha = 1

# Get spam and ham message rows
spam_messages = training_data[training_data["Label"] == "spam"]
ham_messages = training_data[training_data["Label"] == "ham"]

# Calculate probability of getting spam and ham messages
p_spam = len(spam_messages) / len(training_data)
p_ham = len(ham_messages) / len(training_data)

# Calculating nspam and nham
n_spam = spam_messages["SMS"].apply(len).sum()
n_ham = ham_messages["SMS"].apply(len).sum()

# Calculating n_vocabulary
n_vocabulary = len(vocabulary)

print("alpha = {}\np_spam = {}\np_ham = {}\nn_spam = {}\nn_ham = {}\nn_vocabulary = {}".format(alpha, p_spam, p_ham, n_spam, n_ham, n_vocabulary))

alpha = 1
p_spam = 0.13458950201884254
p_ham = 0.8654104979811574
n_spam = 15190
n_ham = 57237
n_vocabulary = 7783


## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [14]:
# Calculation of conditional probability for each word in the vocabulary

p_spam_dict = {}
p_ham_dict = {}

for i, word in enumerate(vocabulary):
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    p_spam_dict[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    p_ham_dict[word] = p_word_given_ham

## Classifying a New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

<ul>
<li>Takes in as input a new message (w1, w2, ..., wn).
<li>Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
<li>Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
<ul shape="round">
<li>If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
<li>If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
<li>If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.
</ul>
</ul>

In [15]:
import re

# Classification function
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_spam_dict:
            p_spam_given_message *= p_spam_dict[word]
        
        if word in p_ham_dict:
            p_ham_given_message *= p_ham_dict[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [16]:
# Let's test the function using some test cases
test_case_1 = "WINNER!! This is the secret code to unlock the money: C3421."
classify(test_case_1)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [17]:
test_case_2 = "Sounds good, Tom, then see u there"
classify(test_case_2)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring Spam Filter's accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [18]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_spam_dict:
            p_spam_given_message *= p_spam_dict[word]

        if word in p_ham_dict:
            p_ham_given_message *= p_ham_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [19]:
# testing the classifier with the test data set
test_data['predicted'] = test_data['SMS'].apply(classify_test_set)
test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [20]:
# Estimating accuracy of the classifier
correct = 0
total = test_data.shape[0]

for row in test_data.iterrows():
    row = row[1]
    
    if row["Label"] == row["predicted"]:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

## Next Steps

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

Next steps include:
<ul>
<li>Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
<li>Make the filtering process more complex by making the algorithm sensitive to letter case
</ul>